## CBoW मोडेल प्रशिक्षण

यो नोटबुक [AI for Beginners Curriculum](http://aka.ms/ai-beginners) को एक भाग हो।

यस उदाहरणमा, हामी CBoW भाषा मोडेल प्रशिक्षण गरेर हाम्रो आफ्नै Word2Vec एम्बेडिङ स्पेस प्राप्त गर्ने प्रयास गर्नेछौं। हामी AG News डेटासेटलाई पाठको स्रोतको रूपमा प्रयोग गर्नेछौं।


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

पहिले हामी हाम्रो डाटासेट लोड गरौं र टोकनाइजर र शब्दकोश परिभाषित गरौं। हामी गणनाहरूलाई केही सीमित गर्न `vocab_size` लाई 5000 मा सेट गर्नेछौं।


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW मोडेल

CBoW ले $2N$ छिमेकी शब्दहरूको आधारमा एउटा शब्दको भविष्यवाणी गर्न सिक्छ। उदाहरणका लागि, जब $N=1$ हुन्छ, हामीले वाक्य *I like to train networks* बाट निम्न जोडीहरू प्राप्त गर्नेछौं: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks)। यहाँ, पहिलो शब्द इनपुटको रूपमा प्रयोग गरिएको छिमेकी शब्द हो, र दोस्रो शब्द भविष्यवाणी गरिएको शब्द हो।

अर्को शब्दको भविष्यवाणी गर्न नेटवर्क निर्माण गर्नका लागि, हामीले छिमेकी शब्दलाई इनपुटको रूपमा दिनुपर्नेछ, र शब्द नम्बरलाई आउटपुटको रूपमा प्राप्त गर्नुपर्नेछ। CBoW नेटवर्कको संरचना निम्न प्रकारको छ:

* इनपुट शब्दलाई embedding layer मार्फत पठाइन्छ। यो embedding layer नै हाम्रो Word2Vec embedding हुनेछ, त्यसैले हामी यसलाई अलग रूपमा `embedder` भेरिएबलको रूपमा परिभाषित गर्नेछौं। यस उदाहरणमा हामी embedding size = 30 प्रयोग गर्नेछौं, यद्यपि तपाईं उच्च dimensions (वास्तविक word2vec मा 300 हुन्छ) प्रयोग गरेर परीक्षण गर्न चाहन सक्नुहुन्छ।
* Embedding vector लाई त्यसपछि एउटा linear layer मा पठाइन्छ जसले आउटपुट शब्दको भविष्यवाणी गर्नेछ। त्यसैले यसमा `vocab_size` neurons हुन्छ।

आउटपुटको लागि, यदि हामी `CrossEntropyLoss` लाई loss function को रूपमा प्रयोग गर्छौं भने, हामीले एक-हट encoding बिना मात्र शब्द नम्बरहरूलाई अपेक्षित परिणामको रूपमा प्रदान गर्नुपर्नेछ।


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## प्रशिक्षण डेटा तयार गर्दै

अब मुख्य कार्यलाई प्रोग्राम गरौं जसले पाठबाट CBoW शब्द जोडीहरू गणना गर्नेछ। यो कार्यले हामीलाई विन्डो साइज निर्दिष्ट गर्न अनुमति दिनेछ, र जोडीहरूको सेट - इनपुट र आउटपुट शब्दहरू फर्काउनेछ। ध्यान दिनुहोस् कि यो कार्य शब्दहरूमा मात्र होइन, भेक्टरहरू/टेन्सरहरूमा पनि प्रयोग गर्न सकिन्छ - जसले हामीलाई पाठलाई एन्कोड गर्न अनुमति दिनेछ, `to_cbow` कार्यमा पठाउनु अघि।


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


आउनुहोस् प्रशिक्षण डेटासेट तयार गरौं। हामी सबै समाचारहरू हेर्नेछौं, `to_cbow` कल गरेर शब्द जोडीहरूको सूची प्राप्त गर्नेछौं, र ती जोडीहरूलाई `X` र `Y` मा थप्नेछौं। समय बचतको लागि, हामी केवल पहिलो १०k समाचार वस्तुहरूलाई विचार गर्नेछौं - यदि तपाईंलाई पर्खनको लागि थप समय छ र राम्रो एम्बेडिङ प्राप्त गर्न चाहनुहुन्छ भने तपाईं सजिलै यो सीमालाई हटाउन सक्नुहुन्छ :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

हामी त्यो डाटालाई पनि एक डेटा सेटमा रूपान्तरण गर्नेछौं, र डाटालोडर सिर्जना गर्नेछौं:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

हामी त्यो डाटालाई पनि एक डाटासेटमा रूपान्तरण गर्नेछौं, र डाटालोडर बनाउनेछौं:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

अब हामी वास्तविक प्रशिक्षण गरौं। हामी `SGD` अप्टिमाइजर उच्च लर्निङ रेटसँग प्रयोग गर्नेछौं। तपाईंले अन्य अप्टिमाइजरहरू, जस्तै `Adam`, प्रयोग गरेर पनि प्रयास गर्न सक्नुहुन्छ। सुरुमा हामी १० इपोक्सको लागि प्रशिक्षण गर्नेछौं - र यदि तपाईं अझ कम हानि चाहनुहुन्छ भने यो सेल पुन: चलाउन सक्नुहुन्छ।


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Word2Vec प्रयोग गर्दै हेर्ने

Word2Vec प्रयोग गर्नको लागि, हाम्रो शब्दकोशमा रहेका सबै शब्दहरूको लागि भेक्टरहरू निकालौं:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


यो Word2Vec प्रयोग गरेर पर्यायवाची शब्दहरू खोज्न रोचक छ। तलको कार्यले दिइएको इनपुटको लागि `n` नजिकका शब्दहरू फिर्ता गर्नेछ। तिनीहरू फेला पार्न, हामी $|w_i - v|$ को मान गणना गर्छौं, जहाँ $v$ हाम्रो इनपुट शब्दसँग सम्बन्धित भेक्टर हो, र $w_i$ शब्दकोशमा रहेको $i$-औं शब्दको इनकोडिङ हो। त्यसपछि हामी एरेलाई क्रमबद्ध गर्छौं र `argsort` प्रयोग गरेर सम्बन्धित सूचकांकहरू फिर्ता गर्छौं, र सूचीका पहिलो `n` तत्वहरू लिन्छौं, जसले शब्दकोशमा नजिकका शब्दहरूको स्थानलाई इनकोड गर्छ।


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## मुख्य कुरा

CBoW जस्ता चतुर तरिकाहरू प्रयोग गरेर, हामी Word2Vec मोडेल प्रशिक्षण गर्न सक्छौं। तपाईंले skip-gram मोडेल पनि प्रशिक्षण गर्न प्रयास गर्न सक्नुहुन्छ, जसले केन्द्रिय शब्द दिइएको अवस्थामा छेउछाउका शब्दहरू अनुमान गर्न सिक्छ, र यसले कत्तिको राम्रो प्रदर्शन गर्छ हेर्न सक्नुहुन्छ।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।  
